Fernando Arturo Arevalo Perez  
Numero de alumno: 323018942  
11. 4. Spark_ML_Koalas

# Ejercicio aplicado_Machine Learning con Spark

En este notebook entrenaremos un modelo de clasificacion binaria capaz de predecir la enfermedad cardiaca en base a diferentes mediciones de parametros bioquimicos. Para ello utilizaremos el dataset de Kaggle

In [1]:
import numpy as np 
import pandas as pd 

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('UCI Heart disease').getOrCreate()

In [5]:
from pyspark.ml.classification import LogisticRegression

### Ejercicio 1  
En este ejercicio vamos a importar y preprocesar los datos de 'data/heart.csv' que utilizaremos para entrenar un modelo de clasificación binaria con PySpark. Para ello, tendrás que inicializar una sesión de Spark, cargar los datos con el esquema correcto y analizar su distribución.  
Es decir, debes completar la parte de importación y análisis exploratorio de los datos.

In [6]:
heart = spark.read.csv('data/heart.csv', 
                       inferSchema = True, 
                       header = True)

In [7]:
heart.show(5)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 5 rows



In [20]:
heart.toPandas().head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [8]:
heart.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- target: integer (nullable = true)



### Ejercicio 2  
En este ejercicio vas a aplicar un pre-procesamiento mínimo en el conjunto de datos del ejercicio anterior, de manera que puedas utilizarlos para entrenar un modelo.

### Preprocesamiento de datos

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
heart.columns

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'target']

In [11]:
assembler = VectorAssembler(
                            inputCols=['age',
                            'sex',
                            'cp',
                            'trestbps',
                            'chol',
                            'fbs',
                            'restecg',
                            'thalach',
                            'exang',
                            'oldpeak',
                            'slope',
                            'ca',
                            'thal'],
                            outputCol="features")

In [12]:
output = assembler.transform(heart)

In [13]:
output.show(5)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|            features|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|[63.0,1.0,3.0,145...|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|[37.0,1.0,2.0,130...|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|[41.0,0.0,1.0,130...|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|[56.0,1.0,1.0,120...|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|[57.0,0.0,0.0,120...|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+--------------------+
only showing top 5 rows



In [14]:
final_data = output.select("features",'target')

### Ejercicio 3  
En este ejercicio vas a entrenar un modelo de clasificación binaria con la librería de machine learning de PySpark, con el conjunto de datos pre-procesados del ejercicio anterior.  
Una vez entrenado el modelo, vas a tener que realizar una predicción con el conjunto de datos de test y comparar los resultados. A continuación, deberás obtener diferentes métricas de evaluación para determinar si el modelo es adecuado o no.

### Entrenamiento del modelo

In [15]:
train, test = final_data.randomSplit([0.7,0.3])

In [16]:
lr = LogisticRegression(labelCol="target",
                        featuresCol="features")

In [21]:
model = lr.fit(train)

In [17]:
predict_train=model.transform(train)
predict_test=model.transform(test)
predict_test.select("target","prediction").show(10)

+------+----------+
|target|prediction|
+------+----------+
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     1|       1.0|
|     0|       0.0|
|     0|       0.0|
|     0|       1.0|
|     1|       1.0|
+------+----------+
only showing top 10 rows



Una vez entrenado el modelo, vas a tener que realizar una predicción con el conjunto de datos de test y comparar los resultados. A continuación, deberás obtener diferentes métricas de evaluación para determinar si el modelo es adecuado o no.

### Evaluación del modelo

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',
                                          labelCol='target')

predict_test.select("target","rawPrediction","prediction","probability").show(5)


+------+--------------------+----------+--------------------+
|target|       rawPrediction|prediction|         probability|
+------+--------------------+----------+--------------------+
|     1|[-1.0007202406120...|       1.0|[0.26879983703912...|
|     1|[-5.4196850873564...|       1.0|[0.00440901552566...|
|     1|[-3.0483944573771...|       1.0|[0.04528683997447...|
|     1|[-3.4004978038762...|       1.0|[0.03227991072058...|
|     1|[-1.6942514754348...|       1.0|[0.15521754762672...|
+------+--------------------+----------+--------------------+
only showing top 5 rows



In [19]:
print("The area under ROC for train set is {}".format(evaluator.evaluate(predict_train)))

print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_test)))

The area under ROC for train set is 0.9352880658436216
The area under ROC for test set is 0.884502923976608


Por lo que en la fase de entrenamiento conseguimos un modelo con un nivel de predicción de entre 90% y 95%.  
Y en la fase de prueba nuestro modelo está dentro de un rango entre 85% y 95%.  
Por lo que se determinaría que el modelo es adecuado. Aunque esto siempre dependrá de la aplicación del modelo.